In [1]:
import numpy as np
from scipy import optimize as opt

In [9]:
# Model Primitives
np.random.seed(1234567890)

nObs = 1000
beta = np.array([0.5, 0.5] , dtype=float)
income= np.random.uniform(size = nObs) # draws from standard normal
explVar = np.vstack([np.ones(nObs), income]).T

In [41]:
def simulateBinaryLogit(x, beta):
    nObs     = x.shape[0]
    nChoice  = 2;
    
    epsilon = np.random.gumbel(size = [nObs, nChoice])
    beta_augmented = np.vstack([np.zeros(beta.shape), beta])
    utility = x @ beta_augmented.T + epsilon
    return np.argmax(utility, axis=1)

In [42]:
choice = simulateBinaryLogit(explVar, beta)

choice[1:10]

array([0, 1, 1, 0, 1, 0, 1, 1, 1], dtype=int64)

In [12]:
data = np.hstack((choice.reshape(nObs, 1), explVar))
data.shape

(1000, 3)

In [13]:
def calcLambda(x, beta):
    prob = np.exp(x @ beta)  / (1 + np.exp(x @ beta))
    return prob

In [110]:
def logLike_binaryLogit(beta, y, x):
    choiceProb   = calcLambda(x, beta);
    
    ll_i         = y * np.log(choiceProb) + (1 - y) * np.log(1 - choiceProb);
    logLike      = -(ll_i.sum())
    return logLike

In [123]:
beta0 = np.zeros(2)
out = opt.minimize(logLike_binaryLogit, beta0, args=(data[:,0], data[:,1:]) , method='L-BFGS-B', tol=1e-12)

print('beta hat is:', out.x)

print('value of likelihood at beta:', out.fun)

# how to get SE's?
out

beta hat is: [ 0.40157312  0.5869323 ]
value of likelihood at beta: 633.200201176


      fun: 633.20020117642594
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 30
      nit: 8
   status: 0
  success: True
        x: array([ 0.40157312,  0.5869323 ])

In [137]:
# Simulated Maximum Likelihood

def logLikeSim_binaryLogit(beta, y, x, nSim):
    np.random.seed(42)
    
    #print(beta)
    #print(y.shape)
    #print(x.shape)
    #print(nSim)
    
    nObs = y.shape[0]
    
    simChoice = np.empty((nObs,nSim))
    simChoice[:] = np.NAN
    
    for iSim in range(0, nSim):
        simChoice[:,iSim] = simulateBinaryLogit(x, beta)
    
    simProb = simChoice.mean(axis=1)
    
    ll_i         = y * np.log(simProb) + (1 - y) * np.log(1 - simProb)
    logLike      = -(ll_i.sum())
    print(logLike)
    return logLike
    


In [133]:
s = logLikeSim_binaryLogit(beta0, data[:,0], data[:,1:],200)

s

[ 0.05  0.05]
(1000,)
(1000, 2)
200
684.020780815


684.02078081502714

In [138]:
beta0 = 0.05*np.ones(2)
nSim = 1000

opt.minimize(logLikeSim_binaryLogit, beta0, args=(data[:,0], data[:,1:], nSim) , method='L-BFGS-B', \
             options={'gtol': 1e-16})

681.536883513
681.536883513
681.536883513


      fun: 681.53688351327219
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 3
      nit: 0
   status: 0
  success: True
        x: array([ 0.05,  0.05])